In [ ]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import h5py
import matplotlib.pyplot as plt
'''import inflect
c2h.stringify = inflect.engine()'''
from tqdm import tqdm
from collections import Counter
import csv_to_hdf5 as c2h
import obspy
import math

In [ ]:
creepmeter_metadata = pd.read_excel('../../Data/Creepmeter_list.xlsx')
creepmeter_metadata

In [ ]:
# Example usage
directory = '../../Data/DATA_tidied/CSV/'  # Replace with your actual directory path
files = c2h.list_files_in_directory(directory)
print(files)
try:
    Creepmeter_dataframe = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation.csv',index_col=0)
except FileNotFoundError:
    Creepmeter_dataframe = pd.DataFrame()

print(Creepmeter_dataframe)
Creepmeter_dataframe_SAC = pd.DataFrame()

In [ ]:
del files[6]
print(files)


In [6]:
files = ['bal1.csv']

In [ ]:
files.sort()
diff_all = []
count = 0
for file in tqdm(files):
    data = pd.read_csv(directory + file,index_col=0)
    try:
        #print(data['Time'].head(10))
        data['Time'] = pd.to_datetime(data['Time'])
    except KeyError:
        #print(data['Date'].head(10))
        data['Time'] = pd.to_datetime(data['Date'])
    tm_diff = np.diff(data['Time'])/ np.timedelta64(1, 'm')
    negatives = np.any(tm_diff<0)
    if negatives == True:
        where_neg = np.where(tm_diff<0)[0]
        count+=1
        print(where_neg)
        for i in range(len(where_neg)):
            print(data.loc[where_neg[i]-10:where_neg[i]+10])
        print(file, 'negatives!!')
    diff_all.extend(np.unique(tm_diff))
print(count)
print(files)

In [14]:
import importlib
importlib.reload(c2h)
try:
    del creeping_A
except NameError:
    dummy=10

try:
    del creeping_B
except NameError:
    dummy=10
try:
    del creeping_C
except NameError:
    dummy=10
sac_vs_hdf5='hdf5'
Creepmeter_dataframe_SAC = pd.DataFrame()
for file in files:
    # Extract the first 4 letters in uppercase
    abbreviation = file[:4].upper()
    print(abbreviation)
    fn = '{k}'.format(k=abbreviation)
    prefix = 'Slip'
    index_CM = np.where(creepmeter_metadata['Creepmeter_abbrv']==abbreviation)[0][0]
    full_name = creepmeter_metadata['Creepmeter_full_name'].iloc[index_CM]
    latitude = creepmeter_metadata['Latitude'].iloc[index_CM]
    longitude = creepmeter_metadata['Longitude'].iloc[index_CM]
    depth = creepmeter_metadata['Depth'].iloc[index_CM]
    length = creepmeter_metadata['Length'].iloc[index_CM]
    obliquity = creepmeter_metadata['Obliquity'].iloc[index_CM]
    network = creepmeter_metadata['Network'].iloc[index_CM]
    full_name = creepmeter_metadata['Creepmeter_full_name'].iloc[index_CM]
    
    hdf5_path = '../../Data/DATA_tidied/HDF5/' + fn +'.h5'
    sac_path = '../../Data/DATA_tidied/SAC/' + fn +'.SAC'

    if sac_vs_hdf5 == 'hdf5':
        path = hdf5_path
    elif sac_vs_hdf5 == 'sac':
        path=sac_path
    print(path)
    if not os.path.isfile(path):
        data = pd.read_csv(directory + file,index_col=0)
        try:
            data['Time'] = pd.to_datetime(data['Time'])
        except KeyError:
            data['Time'] = pd.to_datetime(data['Date'])

        tm_diff = np.diff(data['Time'])/ np.timedelta64(1, 'm')
        tm_diff = np.append(tm_diff,123456)
        tm_diff = c2h.round_to_nearest_half(tm_diff)
        data['tm_diff'] = tm_diff
        data.dropna(inplace=True)
        data['tm_diff'] = c2h.round_to_nearest_half(data['tm_diff'])
        data.reset_index(inplace=True,drop=True)
        

        boolarr = tm_diff <1/60
        same_time = tm_diff[boolarr] 

        if len(same_time) >0:
            datapoint_to_drop = np.where(tm_diff <1/60)[0][0]
            data.drop(datapoint_to_drop,inplace=True)
            data.reset_index(inplace=True)
            data.drop(['index'],axis=1,inplace=True)
            boolarr_diff = tm_diff >= 1/60
            tm_diff = tm_diff[boolarr_diff]
        data.drop(data.loc[data.Time.isnull()].index,inplace=True)
        data.reset_index(inplace=True,drop=True) 
        data['Slip'] = data['Slip'].astype(float)

        plt.figure()
        ax = plt.subplot(1,1,1)
        plt.plot(tm_diff)
        plt.ylim(0,62)
        plt.hlines([1/60,1/6,1/2,1,2,5,10,30,60],xmin=0,xmax=len(tm_diff),colors='orange',linestyles='--',alpha=0.5)
        plt.show()

        allowed_modes = [1/60,1/6,1/2,1,2,5,10,30,60]
        top_4_modes = c2h.find_top_modes(tm_diff, 4,allowed_modes)
        print("Top 4 modes and their frequencies:", top_4_modes)
        no_mode = input('how many sampling frequencies?')
        reversal = input('does the smaple rate revert to an earlier one? (y/n)')
        periods = []
        for q in range(int(no_mode)):
            periods.append(top_4_modes[q][0])
        print(periods)



        if len(periods)==3:
            a = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[0]), g = c2h.stringify(periods[1])))
            b = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[1]), g = c2h.stringify(periods[0])))
            c = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[0]), g = c2h.stringify(periods[2])))
            d = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[2]), g = c2h.stringify(periods[0])))
            e = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[1]), g = c2h.stringify(periods[2])))
            f = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[2]), g = c2h.stringify(periods[1])))

            indices = sorted([a,b,c,d,e,f])
            unique_ind = np.unique(indices)
            creeping_A = data.iloc[:unique_ind[1]]      # From start to 'a' (exclusive)
            the_rest = data.iloc[unique_ind[1]:]
            creeping_B = data.iloc[unique_ind[1]:unique_ind[2]]    # From 'a' (inclusive) to 'b' (exclusive)
            creeping_C = data.iloc[unique_ind[2]:]     # From 'b' (inclusive) to 'c' (exclusive)
            
            creeping_A.reset_index(inplace=True,drop=True)
            creeping_B.reset_index(inplace=True,drop=True)
            creeping_C.reset_index(inplace=True,drop=True)
            
            print(creeping_A)
            print(creeping_B)
            print(creeping_C)


        elif len(periods) ==2:
            a = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[0]), g = c2h.stringify(periods[1])))
            b = eval('c2h.{k}_to_{g}(tm_diff)'.format(k = c2h.stringify(periods[1]), g = c2h.stringify(periods[0])))

            indices = sorted([a,b])
            unique_ind = np.unique(indices)
            creeping_A = data.iloc[:unique_ind[1]]
            the_rest = data.iloc[unique_ind[1]:]
            
            if reversal == 'y':
                the_rest.reset_index(inplace=True,drop=True)
                a = eval('c2h.{k}_to_{g}(the_rest["tm_diff"])'.format(k = c2h.stringify(periods[0]), g = c2h.stringify(periods[1])))
                b = eval('c2h.{k}_to_{g}(the_rest["tm_diff"])'.format(k = c2h.stringify(periods[1]), g = c2h.stringify(periods[0])))
                print(a,b)
                if abbreviation == 'R7' or abbreviation == 'RB':
                    a = 17179
                indices = np.unique([a,b])
                print(indices)

                creeping_B = the_rest.iloc[indices[0]:indices[1]]
                creeping_C  = the_rest.iloc[indices[1]:]
                
                creeping_A = pd.concat([creeping_A,creeping_C],ignore_index=True)
                creeping_B.reset_index(inplace=True,drop=True)
                del creeping_C

            else:
                creeping_B = data.iloc[unique_ind[1]:]
                creeping_A.reset_index(inplace=True,drop=True)
                creeping_B.reset_index(inplace=True,drop=True)

            print(creeping_A)
            print(creeping_B)

        elif len(periods) == 1:
            creeping_A = data.iloc[:]
            print(creeping_A)
        
        
        Number_of_splits = []
        Sampling_rates_variable = []
        try:
            sampling_A = c2h.find_top_modes(creeping_A['tm_diff'],1,allowed_modes)[0][0]
            Sampling_rates_variable.append(sampling_A)
            Number_of_splits.append('A')
        except (IndexError, NameError, KeyError):
            sampling_A = 0

        try:
            sampling_B = c2h.find_top_modes(creeping_B['tm_diff'],1,allowed_modes)[0][0]
            Sampling_rates_variable.append(sampling_B)
            Number_of_splits.append('B')
        except (IndexError, NameError, KeyError):
            sampling_B = 0

        try:
            sampling_C = c2h.find_top_modes(creeping_C['tm_diff'],1,allowed_modes)[0][0]
            Sampling_rates_variable.append(sampling_C)
            Number_of_splits.append('C')
        except (IndexError, NameError, KeyError):
            sampling_C = 0

        print(Sampling_rates_variable,Number_of_splits)

        if sac_vs_hdf5 == 'sac':
            for i in range(len(Number_of_splits)):
                print('interpolating creeping_{k}'.format(k=Number_of_splits[i]))
                upsampled = eval('creeping_{k}'.format(k=Number_of_splits[i]))
                val1_okay = math.isnan(upsampled.Slip.iloc[0])
                if val1_okay == True:
                    upsampled.reset_index(inplace=True)
                    upsampled.drop([0],axis=0,inplace=True)
                tr = obspy.Trace(np.array(upsampled.Slip))
                st = obspy.Stream(tr)
                st[0].stats.network = network
                st[0].stats.station = abbreviation
                st[0].stats.location = '00'
                st[0].stats.channel = 'slip'
                st[0].stats.starttime = pd.to_datetime(upsampled.Time.iloc[0])
                st[0].stats.delta = 60*Sampling_rates_variable[i]
                #st[0].plot()
                instrument = pd.DataFrame({'Network':[st[0].stats.network],'Creepmeter_full_name':['{k}'.format(k=full_name)],
                                        'Creepmeter_abbrv':['{k}'.format(k=abbreviation)],'File_code':['{p}_{q}'.format(p=abbreviation,q=i)],
                                        'Start Time':[st[0].stats.starttime],'End Time':[st[0].stats.endtime],
                                        'Sampling rate, Hz':[st[0].stats.sampling_rate],'Sampling rate, mins':(1/st[0].stats.sampling_rate)/60,
                                        'Latitude':[latitude],'Longitude':[longitude]})
                
                Creepmeter_dataframe_SAC = pd.concat([Creepmeter_dataframe_SAC,instrument],ignore_index=True)
                Creepmeter_dataframe_SAC.drop_duplicates(subset='File_code',inplace=True)
                Creepmeter_dataframe_SAC.reset_index(inplace=True,drop=True)
                
                Creepmeter_dataframe_SAC.to_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes.csv')
                path = '../../Data/DATA_tidied/SAC/'
                st.write(path +'{p}_{q}.SAC'.format(p=abbreviation,q=i),format='SAC')
                st[0].plot();
            print(Creepmeter_dataframe_SAC)
            try:
                del creeping_A
            except NameError:
                dummy=10

            try:
                del creeping_B
            except NameError:
                dummy=10
            try:
                del creeping_C
            except NameError:
                dummy=10
        
        elif sac_vs_hdf5 =='hdf5':
            with h5py.File(path, 'w') as f:
                f.attrs['author'] = 'Daniel B. Gittins'
                f.attrs['network'] = network
                f.attrs['latitude'] = latitude
                f.attrs['longitude'] = longitude
                f.attrs['depth'] = depth
                f.attrs['length'] = length
                f.attrs['obliquity'] = obliquity
                # Create a group to store both waves together
                for i in range(len(Number_of_splits)):
                    smpl_rate = Sampling_rates_variable[i]
                    group_name = fn+'_{k}mins'.format(k=smpl_rate)
                    time_data = eval('creeping_{k}.Time'.format(k=Number_of_splits[i]))
                    # Assuming 'data['Time']' is your pandas Series of datetime objects
                    datetime_strings = time_data.dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

                    dependent_data = eval('creeping_{k}.Slip'.format(k=Number_of_splits[i])).tolist()
                    group = f.create_group(group_name)
                    # Create datasets for time and dependent variable in the group
                    group.create_dataset('Time_{p}_{k}mins'.format(p=abbreviation,k=smpl_rate), data=datetime_strings)
                    group.create_dataset('Slip_{p}_{k}mins'.format(p=abbreviation,k=smpl_rate), data=dependent_data)
                
                    
                    # Store metadata about the dependent variable
                    if Sampling_rates_variable[i] >=1:
                        description = f"Slip for {abbreviation} at {smpl_rate} minute sampling"
                    else:
                        description = f"Slip for {abbreviation} at {smpl_rate*60} second sampling"
                    group.attrs['description'] = description

                    
                    # Optional: Store additional metadata (e.g., units, sampling rate)
                    starttime = eval('creeping_{k}.Time.iloc[0]'.format(k=Number_of_splits[i]))
                    endtime = eval('creeping_{k}.Time.iloc[-1]'.format(k=Number_of_splits[i]))
                    group.attrs['sampling_rate'] = Sampling_rates_variable[i]  # Example: 100 Hz sampling rate (10 ms interval)
                    group.attrs['time_units'] = 'minutes'     # Time units
                    group.attrs['slip_units'] = 'millimetres'  # Dependent variable units (e.g., for slip)
                    group.attrs['channel'] = 'slip'
                    group.attrs['starttime'] = starttime.strftime('%Y-%m-%d %H:%M:%S')
                    group.attrs['delta'] = 60*Sampling_rates_variable[i]
                    instrument = pd.DataFrame({'Network':[network],'Creepmeter_full_name':[full_name],'Creepmeter_abbrv':[abbreviation],'File_code':[fn],
                                            'Start Time':[starttime],'End Time':[endtime],'Sampling rate, mins':[smpl_rate],
                                            'Latitude':[latitude],'Longitude':[longitude],'Description':[description],'Depth':[depth],'Length':[length],
                                            'Obliquity':[obliquity],'group_name':group_name})
                    Creepmeter_dataframe = pd.concat([Creepmeter_dataframe,instrument],ignore_index=True)
                try:
                    del creeping_A
                except NameError:
                    dummy=10

                try:
                    del creeping_B
                except NameError:
                    dummy=10
                try:
                    del creeping_C
                except NameError:
                    dummy=10
            print('HDF5 structure')
            c2h.print_hdf5_structure('../../Data/DATA_tidied/HDF5/' + fn +'.h5')
            Creepmeter_dataframe.drop_duplicates()
            Creepmeter_dataframe.reset_index(inplace=True,drop=True)
            Creepmeter_dataframe.to_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation.csv')
            print('Creepmeter_meta_data')
            print(Creepmeter_dataframe)




BAL1
../../Data/DATA_tidied/HDF5/BAL1.h5
C461
../../Data/DATA_tidied/HDF5/C461.h5
C462
../../Data/DATA_tidied/HDF5/C462.h5
CAN1
../../Data/DATA_tidied/HDF5/CAN1.h5
CER1
../../Data/DATA_tidied/HDF5/CER1.h5
CFW1
../../Data/DATA_tidied/HDF5/CFW1.h5
CHA1
../../Data/DATA_tidied/HDF5/CHA1.h5
CHE1
../../Data/DATA_tidied/HDF5/CHE1.h5
CHP1
../../Data/DATA_tidied/HDF5/CHP1.h5
COZ1
../../Data/DATA_tidied/HDF5/COZ1.h5
CPP1
../../Data/DATA_tidied/HDF5/CPP1.h5
CRR1
../../Data/DATA_tidied/HDF5/CRR1.h5
CTM1
../../Data/DATA_tidied/HDF5/CTM1.h5
CWC3
../../Data/DATA_tidied/HDF5/CWC3.h5
CWN1
../../Data/DATA_tidied/HDF5/CWN1.h5
ESZ1
../../Data/DATA_tidied/HDF5/ESZ1.h5
FXC1
../../Data/DATA_tidied/HDF5/FXC1.h5
GB.C


IndexError: index 0 is out of bounds for axis 0 with size 0

Adding Temperature

In [200]:
Temp = pd.read_csv('../../Data/DATA_tidied/CSV/Temperature/Balikburnu_Temp.csv',index_col=0)

In [201]:
Temp['Date'] = pd.to_datetime(Temp['Date'])

In [202]:
abbreviation = 'BAL1'

In [ ]:
time_data = Temp['Date']
datetime_strings = time_data.dt.strftime('%Y-%m-%d %H:%M:%S').tolist()
dependent_data = Temp['degC'].tolist()
hdf5_path = '../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbreviation)
with h5py.File(hdf5_path, 'a') as f:
    del f['Temperature']
    group = f.create_group('Temperature')
    # Create datasets for time and dependent variable in the group
    group.create_dataset('Time_{p}'.format(p=abbreviation), data=datetime_strings)
    group.create_dataset('Temperature_{p}'.format(p=abbreviation), data=dependent_data)
    group.attrs['Units'] = 'Celsius'
print('HDF5 structure')
c2h.print_hdf5_structure(hdf5_path)

Daily

In [ ]:
Daily = pd.read_csv('../../Data/DATA_tidied/CSV/Daily/xva1_day.csv',index_col=0)
Daily

In [284]:
abbreviation = 'XVA1'

In [ ]:
datetime_strings = Daily['Time'].tolist()
dependent_data = Daily['Slip'].tolist()
hdf5_path = '../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbreviation)
with h5py.File(hdf5_path, 'a') as f:
    #del f['Daily measurements']
    group = f.create_group('Daily_measurements')
    # Create datasets for time and dependent variable in the group
    group.create_dataset('Time_{p}_daily'.format(p=abbreviation), data=datetime_strings)
    group.create_dataset('Slip_{p}_daily'.format(p=abbreviation), data=dependent_data)
    group.attrs['Units_slip'] =  'millimetres'
    group.attrs['Start date'] = datetime_strings[0]
    group.attrs['End date'] = datetime_strings[-1]
print('HDF5 structure')
c2h.print_hdf5_structure(hdf5_path)

Manual

In [ ]:
Manual = pd.read_csv('../../Data/DATA_tidied/CSV/Manual/bit1_man.csv',index_col=0)
Manual

In [533]:
abbreviation = 'BIT1'

In [ ]:
datetime_strings = Manual['Time'].tolist()
dependent_data = Manual['Slip'].tolist()
hdf5_path = '../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbreviation)
with h5py.File(hdf5_path, 'a') as f:
    del f['Manual_measurements']
    group = f.create_group('Manual_measurements')
    # Create datasets for time and dependent variable in the group
    group.create_dataset('Time_{p}_Manual'.format(p=abbreviation), data=datetime_strings)
    group.create_dataset('Slip_{p}_Manual'.format(p=abbreviation), data=dependent_data)
    group.attrs['Units_slip'] =  'millimetres'
    group.attrs['Start date'] = datetime_strings[0]
    group.attrs['End date'] = datetime_strings[-1]
print('HDF5 structure')
c2h.print_hdf5_structure(hdf5_path)

Orthogonal

In [ ]:
Ortho = pd.read_csv('../../Data/DATA_tidied/CSV/Orthogonal/Karakose_Orthogonal.csv',index_col=0)
Ortho['Date'] = pd.to_datetime(Ortho['Date'])
tm_diff = np.diff(Ortho['Date'])/ np.timedelta64(1, 'm')
Ortho['diff'] = np.append(tm_diff,123456)
Ortho

In [544]:
abbreviation = 'KAR1'

In [ ]:
time_data = Ortho['Date']
datetime_strings = time_data.dt.strftime('%Y-%m-%d %H:%M:%S').tolist()
dependent_data = Ortho['Ortho'].tolist()
hdf5_path = '../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbreviation)
with h5py.File(hdf5_path, 'a') as f:
    #del f['Orthogonal']
    group = f.create_group('Orthogonal')
    # Create datasets for time and dependent variable in the group
    group.create_dataset('Time_{p}_Orthogonal'.format(p=abbreviation), data=datetime_strings)
    group.create_dataset('Slip_{p}_Orthogonal'.format(p=abbreviation), data=dependent_data)
    group.attrs['Units_Orthogonal'] =  'millimetres'
    group.attrs['Start date'] = datetime_strings[0]
    group.attrs['End date'] = datetime_strings[-1]
print('HDF5 structure')
c2h.print_hdf5_structure(hdf5_path)

HDF5 to SAC

Key: CWN1_1.0mins
Data: <HDF5 group "/CWN1_1.0mins" (2 members)>
Slip: [389.44025 389.43839 389.43839 ... 424.91962 424.91962 424.92059]
Time: DatetimeIndex(['2020-09-22 13:52:00', '2020-09-22 13:52:59',
               '2020-09-22 13:54:00', '2020-09-22 13:54:59',
               '2020-09-22 13:56:00', '2020-09-22 13:57:00',
               '2020-09-22 13:57:59', '2020-09-22 13:59:00',
               '2020-09-22 13:59:59', '2020-09-22 14:01:00',
               ...
               '2023-12-15 12:51:00', '2023-12-15 12:51:59',
               '2023-12-15 12:53:00', '2023-12-15 12:54:00',
               '2023-12-15 12:54:59', '2023-12-15 12:56:00',
               '2023-12-15 12:56:59', '2023-12-15 12:58:00',
               '2023-12-15 12:58:59', '2023-12-15 13:00:00'],
              dtype='datetime64[ns]', length=1573217, freq=None)
<Attributes of HDF5 object at 140592647257872>
Key: CWN1_10.0mins
CWN1_10.0mins not a key, if daily or manual do not worry.
Data: <HDF5 group "/CWN1_10.0mins" (2 

Code below is for importing into Igor

In [ ]:
#pragma TextEncoding = "UTF-8"
#pragma rtGlobals=3
#pragma DefaultTab={3,20,4}

// Creates a date/time wave from a date wave stored as text in YYYY-MM-DD format
// and a time wave stored as text in HH:MM:YY format.
// Example:
//      Make/O/T wDateAsText = {"2016-08-15", "2016-08-15"}
//      Make/O/T wTimeAsText = {"19:01:00", "19:01:18"}
//      TextWavesToDateTimeWave(wDateAsText, wTimeAsText, "wDateTime")
//      Edit wDateTime

Function ConvertTextToDateTime(dateTimeAsText)
    String dateTimeAsText       // Assumed in YYYY-MM-DD HH:MM:SS format
    
    Variable dt
    Variable year, month, day
    Variable hour, minute, second

    // Use sscanf to parse the full datetime string
    sscanf dateTimeAsText, "%d-%d-%d %d:%d:%d", year, month, day, hour, minute, second
    
    // Convert the date part to seconds since the epoch
    dt = Date2Secs(year, month, day)
    
    // Calculate the time of day in seconds
    Variable timeOfDay = 3600 * hour + 60 * minute + second
    
    // Add the time of day to the date
    dt += timeOfDay
    
    return dt
End


Function/WAVE TextWavesToDateTimeWave(dateTimeAsTextWave, outputWaveName)
    WAVE/T dateTimeAsTextWave       // Assumed in YYYY-MM-DD HH:MM:SS format
    String outputWaveName

    Variable numPoints = numpnts(dateTimeAsTextWave)
    Make/O/D/N=(numPoints) $outputWaveName
    WAVE wOut = $outputWaveName
    SetScale d, 0, 0, "dat", wOut
   
    Variable i
    for(i=0; i<numPoints; i+=1)
        String dateTimeAsText = dateTimeAsTextWave[i]
        Variable dt = ConvertTextToDateTime(dateTimeAsText)
        wOut[i] = dt   
    endfor 

    return wOut
End